 # <font color = #254117>[EEP 147]: ESG Analysis Round 3</font>

<div style="width:image width px; font-size:80%; text-align:center;"><img src="big_creek.jpg" alt="alternate text" width="500" height="height" style="padding-bottom:0.5em;" />Big Creek Hydroelectric Project - Southern California Edison</div>

This notebook can be utilized for analysis of the Electricity Strategy Game. This notebook builds on the Round 0 notebook by allowing carbon prices to be adjusted in more of the function, including separately for each of Rounds 4, 5, and 6 in the NPV of expected profits function.

First on our agenda is to import **<font color = ##008700>dependencies</font>** -- packages in Python that add to the basic functions in Python -- same as before.

In [ ]:
from datascience import *
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, Dropdown, IntSlider
import ipywidgets as widgets
from functools import partial
from IPython.display import display
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [10,6]

The variable **current_period** should contain the current round.

The variable **pab_periods** should contain each of the periods for which there was or will be a pay-as-bid auction.  This shouldn't change.

In [ ]:
current_period = 4
pab_periods = [1]

Next we import Demand (realized and forecasted), Bids, Porfolios, and the Auction results.

In [ ]:
demand_table = Table.read_table('demand.csv')
bids_mc = Table.read_table('Bids/MC_bids.csv').sort('PORTFOLIO')
ESG = Table.read_table('ESGPorfolios.csv')
auction_results = Table.read_table('portfolio_auction.csv')

In the following cell we will join the tables based on the column **Plant_ID**.  We will incorporate the actual bids of the rounds completed.

In [ ]:
def get_bids(section):
    bids_all = bids_mc.copy()
    if current_period > 1:
        bids_all_df = bids_all.sort("PLANT_ID").to_df()
        bids_actual = Table.read_table('Bids/' + section + '_bids_' + str(current_period - 1) + '.csv').sort('PORTFOLIO')
        bids_actual_df = bids_actual.sort("PLANT_ID").to_df()
        for period_i in range(1,current_period):
            bids_all_df.loc[bids_all_df["PERIOD"] == period_i] = bids_actual_df.loc[bids_actual_df["PERIOD"] == period_i].values
        bids_all = Table.from_df(bids_all_df)
        
    joined_table_all = bids_all.join("PLANT_ID", ESG, "Plant_ID").sort("PLANT_ID")
    return(joined_table_all)

Define helper functions

In [ ]:
energy_colors_dict = {'Bay Views' : '#EC5F67', 'Beachfront' : '#F29056', 'Big Coal' : '#F9C863', 'Big Gas' : '#99C794', 
                      'East Bay' : '#5FB3B3', 'Fossil Light' : '#6699CC', 'Old Timers' : '#C594C5'}

def demand_calc(hour, period, demand_change):
    demand = demand_table.where("round", period).where("hour", hour)["load"].item()
    if demand_change == 0:
        demand = demand
    elif np.abs(demand_change) < 1:
        demand *= (1 + demand_change)
    else:
        demand += demand_change
    return(demand)

def price_calc(input_table, demand, hour, period):
    #hour and period determine which bids are taken from joined_table
    joined_table = input_table.copy()
    sorted_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    price = 0
    sum_cap = 0
    for i in range(0,len(sorted_table['Capacity_MW'])):
        if sum_cap + sorted_table['Capacity_MW'][i] > demand:
            price = sorted_table['PRICE' + str(hour)][i]
            break
        else:
            sum_cap += sorted_table['Capacity_MW'][i]
            price = sorted_table['PRICE' + str(hour)][i]
    return price

def find_x_pos(widths):
    cumulative_widths = [0]
    cumulative_widths.extend(np.cumsum(widths))
    half_widths = [i/2 for i in widths]
    x_pos = []
    for i in range(0, len(half_widths)):
        x_pos.append(half_widths[i] + cumulative_widths[i])
    return x_pos

def price_line_plot(price):
    plt.axhline(y=price, color='r', linewidth = 2)
    
def demand_plot(demand):
    plt.axvline(x=demand, color='r', linewidth = 2)
    
def adjust_bids_by_cp(input_table, hour, period, carbon_price):
    joined_table = input_table.copy()
    joined_table["Total_Var_Cost_USDperMWH"] += carbon_price * joined_table["Carbon_tonsperMWH"]
    joined_table["PRICE" + str(hour)] += carbon_price * joined_table["Carbon_tonsperMWH"]
    return(joined_table)
    
def user_defined_bids(input_table, hour, period, my_portfolio, def_my_bids, def_others_bids):
    joined_table = input_table.copy()
    joined_df = joined_table.to_df()
    if def_my_bids:
        joined_df.loc[(joined_df["Group"] == my_portfolio) & 
                      (joined_df["PERIOD"] == period), 
                      "PRICE" + str(hour)] =  list(globals()["bids_" + my_portfolio.replace(" ", "").lower()].values())
    if def_others_bids:
        for group in set(joined_table['Group']):
            if group != my_portfolio:
                joined_df.loc[(joined_df["Group"] == group) & 
                              (joined_df["PERIOD"] == period), 
                              "PRICE" + str(hour)] =  list(globals()["bids_" + group.replace(" ", "").lower()].values())
    joined_table = Table.from_df(joined_df)
    return(joined_table)

In [ ]:
def profit_calc(input_table, hour, period, demand, price, my_portfolio):
    if period in pab_periods:
        return(profit_pab(input_table, hour, period, demand, price, my_portfolio))
    
    sorted_joined_table = input_table.copy()
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    capacity_subset = sum(sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    capacity_subset += sum(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"] * marg_proportion)
    revenue = capacity_subset * price
    cost = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return revenue - cost

def profit_pab(input_table, hour, period, demand, price, my_portfolio):    
    sorted_joined_table = input_table.copy()
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    revenue = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))['PRICE' + str(hour)])):
        revenue += sorted_table.where('PRICE' + str(hour), are.below(price))['PRICE' + str(hour)][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))['PRICE' + str(hour)])):
        revenue += sorted_table.where('PRICE' + str(hour), are.equal_to(price))['PRICE' + str(hour)][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    cost = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return revenue - cost

In [ ]:
def emissions_calc(input_table, hour, period, demand, price, my_portfolio):    
    sorted_joined_table = input_table.copy()
    
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    sorted_table = sorted_joined_table.where("Group", my_portfolio)
    emissions = 0
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.below(price))["Total_Var_Cost_USDperMWH"])):
        emissions += sorted_table.where('PRICE' + str(hour), are.below(price))["Carbon_tonsperMWH"][i]\
        * sorted_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"][i]
    for i in range(len(sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Total_Var_Cost_USDperMWH"])):
        emissions += sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Carbon_tonsperMWH"][i]\
        * (sorted_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"][i] * marg_proportion)
    return emissions

In [ ]:
def market_plot(input_table, hour, period, demand, price):
    sorted_joined_table = input_table.copy()
    
    width = sorted_joined_table.column("Capacity_MW")
    height = sorted_joined_table.column('PRICE' + str(hour))
    x_vals = find_x_pos(width)
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(energy_colors_dict))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    
    #prepare the Marginal Cost to be a dashed line
    num_plants = len(width)
    height_mc = sorted_joined_table.column("Total_Var_Cost_USDperMWH")
    x_vec = np.zeros(num_plants * 2)
    h_vec = np.zeros(num_plants * 2)
    for i, (w, h) in enumerate(zip(width, height_mc)):
        h_vec[2*i] = h
        h_vec[2*i+1] = h
        if i == 0:
            x_vec[1] = w
        else: 
            x_vec[2*i] = x_vec[2*i - 1]
            x_vec[2*i + 1] = x_vec[2*i] + w
            
    # Make the plot
    plt.figure(figsize=(9,6))
    plt.bar(x_vals, height, width=width, color=sorted_joined_table['Color'], edgecolor = "black")
    line_mc = plt.plot(x_vec, h_vec, '--k', label='Marginal Cost', linewidth=2)
    patches = []
    for row in group_colors.rows:
        patches += [mpatches.Patch(color=row.item("Color"), label=row.item("Group"))]    
    patches += line_mc
    plt.legend(handles=patches, bbox_to_anchor=(1.1,1))
    plt.title('Energy Market')
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    price_line_plot(price)
    demand_plot(demand)
    plt.show()

In [ ]:
def portfolio_plot(input_table, hour, period, demand, price, my_portfolio):    
    sorted_joined_table = input_table.copy()
    
    your_source = sorted_joined_table.where("Group", my_portfolio)
    width_yours = your_source.column("Capacity_MW")
    height_yours = your_source.column('PRICE' + str(hour))
    new_x_yours = find_x_pos(width_yours)
    label_yours = your_source.column("PLANT")
    colors_mapped = list(pd.Series(sorted_joined_table['Group']).map(energy_colors_dict))
    sorted_joined_table = sorted_joined_table.with_column('Color', colors_mapped)
    group_colors = sorted_joined_table.group("Group", lambda x: x).select("Group", "Color")
    group_colors["Color"] = group_colors.apply(lambda x: x[0], "Color")
    
    #prepare the Marginal Cost to be a dashed line
    num_plants = len(width_yours)
    height_mc = your_source.column("Total_Var_Cost_USDperMWH")
    x_vec = np.zeros(num_plants * 2)
    h_vec = np.zeros(num_plants * 2)
    for i, (w, h) in enumerate(zip(width_yours, height_mc)):
        h_vec[2*i] = h
        h_vec[2*i+1] = h
        if i == 0:
            x_vec[1] = w
        else: 
            x_vec[2*i] = x_vec[2*i - 1]
            x_vec[2*i + 1] = x_vec[2*i] + w
            
    # Make the plot
    plt.figure(figsize=(11,6))
    plt.bar(new_x_yours, height_yours, width=width_yours, 
            color = energy_colors_dict[my_portfolio], edgecolor = "black")
    line_mc = plt.plot(x_vec, h_vec, '--k', label='Marginal Cost', linewidth=2)
    plt.title("Bids: " + my_portfolio)
    plt.xlabel('Capacity_MW')
    plt.ylabel('Price')
    for new_x_i, height_i, label_i in zip(new_x_yours, height_yours, label_yours):
        plt.text(new_x_i, height_i, label_i, ha='center', va='bottom', fontsize=8)
    price_line_plot(price)
            
    #the marginal plants should indicate how much capacity they produce
    nonmarg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.below(price))["Capacity_MW"])
    marg_capacity = sum(sorted_joined_table.where('PRICE' + str(hour), are.equal_to(price))["Capacity_MW"])
    marg_demand = demand - nonmarg_capacity
    marg_proportion = marg_demand / marg_capacity
    
    curr_capacity = 0
    
    for i, (w, h) in enumerate(zip(width_yours, height_yours)):
        if h == price:
            x_val = curr_capacity + (w * marg_proportion)
            x_vec = [x_val, x_val]
            h_vec = [0, h]
            plt.plot(x_vec, h_vec, '--k', linewidth=1)
        elif h >= price:
            break
        curr_capacity += w
    
    plt.axvline(x=curr_capacity, color='r', linewidth = 2)
    
    plt.show()

In [ ]:
def total_profits(section, my_portfolio):
    # Merge auction results with input table
    portfolio_profit_dict = {}
    auction_results_section = auction_results.where("world_id", section)
    if current_period > 1:
        joined_table = get_bids(section)
        full_table = joined_table.join("TEAM", auction_results_section, "team")
        portfolio_table = full_table.where("Group", my_portfolio)
        portfolio_profit = -portfolio_table.where("PERIOD", 1).to_df().loc[0, "adjustment"] * 1.05**(current_period-2)
        for period_i in range(1, current_period):        
            portfolio_table_period = joined_table.where("PERIOD", period_i).where("Group", my_portfolio)
            portfolio_profit_period = -sum(portfolio_table_period["FixedCst_OandM_perDay"])
            
            for hour_i in range(1, 5):
                demand = demand_calc(hour_i, period_i, 0)
                sorted_joined_table = joined_table.where("PERIOD", period_i).sort("PRICE" + str(hour_i), descending = False)
                price = price_calc(sorted_joined_table, demand, hour_i, period_i)
                
                portfolio_profit_period += profit_calc(sorted_joined_table, hour_i, period_i, demand, price, my_portfolio)
                portfolio_profit_dict['Round ' + str(period_i)] = portfolio_profit_period
                    
            portfolio_profit += portfolio_profit_period * 1.05**(current_period - period_i - 1)
    else: 
        portfolio_profit = 0
    portfolio_profit_dict['Total'] = portfolio_profit
    
    output_df = pd.DataFrame.from_dict(portfolio_profit_dict, 
                                       orient = 'index', columns = [my_portfolio + ' Profit']).round().astype(int)
    
    return output_df

In [ ]:
def total_emissions(section, my_portfolio):
    portfolio_emissions_dict = {}
    if current_period > 1:
        joined_table = get_bids(section)
        portfolio_emissions = 0
        for period_i in range(1, current_period):        
            portfolio_emissions_period = 0
            
            for hour_i in range(1, 5):
                demand = demand_calc(hour_i, period_i, 0)
                sorted_joined_table = joined_table.where("PERIOD", period_i).sort("PRICE" + str(hour_i), descending = False)
                price = price_calc(sorted_joined_table, demand, hour_i, period_i)
                
                portfolio_emissions_period += emissions_calc(sorted_joined_table, hour_i, period_i, demand, price, my_portfolio)
                portfolio_emissions_dict['Round ' + str(period_i)] = portfolio_emissions_period
                    
            portfolio_emissions += portfolio_emissions_period
    else: 
        portfolio_emissions = 0
    portfolio_emissions_dict['Total'] = portfolio_emissions
    
    output_df = pd.DataFrame.from_dict(portfolio_emissions_dict, 
                                       orient = 'index', columns = [my_portfolio + ' Emissions']).round().astype(int)
    
    return output_df

Here is the main wrapper function

In [ ]:
def all_output(section, hour, period, demand_change, carbon_price, my_portfolio, def_my_bids, def_others_bids):
    demand = demand_calc(hour, period, demand_change)
    print("Demand: " + str(demand))
    
    joined_table = get_bids(section)
    if period >= current_period:
        joined_table = adjust_bids_by_cp(joined_table, hour, period, carbon_price)
    joined_table = user_defined_bids(joined_table, hour, period, my_portfolio, def_my_bids, def_others_bids)
    sorted_joined_table = joined_table.where("PERIOD", period).sort("PRICE" + str(hour), descending = False)
    price = price_calc(sorted_joined_table, demand, hour, period)
    print("Price: " + str(price))
    
    my_profit = profit_calc(sorted_joined_table, hour, period, demand, price, my_portfolio)
    print('Profit: $' + str(round(my_profit, 2)))
    my_emissions = emissions_calc(sorted_joined_table, hour, period, demand, price, my_portfolio)
    print('Emissions: ' + str(round(my_emissions, 2)) + ' Tons CO2')
    
    market_plot(sorted_joined_table, hour, period, demand, price)
    portfolio_plot(sorted_joined_table, hour, period, demand, price, my_portfolio)
    
    profits_df = total_profits(section, my_portfolio)
    emissions_df = total_emissions(section, my_portfolio)
    display_df = pd.merge(profits_df, emissions_df, left_index=True, right_index=True)
    display(display_df)

In the next cell, we can define the bids.

In [ ]:
bids_bigcoal = {'fourcorners' : 36.5, 'alamitos7' : 73.72, 'huntingtonbeach1_2' : 40.5, 'huntingtonbeach5' : 66.5, 
                         'redondo5_6' : 41.94, 'redondo7_8' : 41.94}

bids_biggas = {'elsegundo1_2' : 44.83, 'elsegundo3_4' : 41.22, 'longbeach' : 52.5, 'northisland' : 65.5, 
               'encina' : 41.67, 'kearny' : 90.06, 'southbay' : 43.83}

bids_bayviews = {'morrobay1_2' : 38.78, 'morrobay3_4' : 36.61, 'mosslanding6' : 32.56, 'mosslanding7' : 32.56, 
                 'oakland' : 61.17}

bids_beachfront = {'coolwater' : 300.02, 'etiwanda1_4' : 300.03, 'etiwanda5' : 300.05, 'ellwood' : 300.06, 'mandalay1_2' : 300.01,
                   'mandalay3' : 300.04, 'ormondbeach1' : 300, 'ormondbeach2' : 300}

bids_eastbay = {'pittsburgh1_4' : 40.94, 'pittsburgh5_6' : 36.61, 'pittsburgh7' : 59.72, 'contracosta4_5' : 58.28, 
                         'contracosta6_7' : 39.5, 'potrerohill' : 69.83}

bids_oldtimers = {'bigcreek' : 0, 'mohave1' : 34.5, 'mohave2' : 34.5, 'highgrove' : 49.61, 'sanbernadino' : 53.94}

bids_fossillight = {'humboldt' : 47.44, 'helms' : 0.5, 'hunterspoint1_2' : 49.17, 'hunterspoint4' : 75.89, 
                    'diablocanyon1' : 11.5}

The next cell runs everything.

Assign **section** to the section code (including quotes) that corresponds to your own according to the following table.

| Code | Section Time  |
|---------|---------------|
| "W8" | Wednesday 8am |
| "W9" | Wednesday 9am |
| "F2" | Friday 2pm |
| "F3" | Friday 3pm |

In [ ]:
interact(lambda section, hour, period, my_portfolio, def_my_bids, def_others_bids:
         all_output(section = section,
                    hour = hour,
                    period = period,
                        #demand_change = 0 uses realized demand for past rounds, forecasted demand for future rounds
                        #abs(demand_change) < 1 will use a percent change in demand 
                        # (e.g. demand_change = -.03 will cause a 3% reduction in demand by multiplying demand by .97)
                        #abs(demand_change) >= 1 will use an absolute change in demand 
                        # (e.g. demand_change = 10 will add 10 to demand)
                    demand_change = 0,
                        #Changing the carbon price will automatically adjust MC.
                    carbon_price = 0,
                    my_portfolio = my_portfolio,
                    def_my_bids = def_my_bids,
                    def_others_bids = def_others_bids),
         section = Dropdown(options=['W8','W9','F2','F3']),
         hour = Dropdown(options=list(range(1,5))), 
         period = Dropdown(options=list(range(1,7))),
         my_portfolio = Dropdown(options=np.unique(ESG["Group"])),
         def_my_bids = Dropdown(options=[False, True]),
         def_others_bids = Dropdown(options=[False, True]))
print('')

Thanks for help from: Alec Kan, Alma Pineda, Aarish Irfan, Elaine Chien, Octavian Sima, and Eric Van Dusen.